In [3]:
%matplotlib inline
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from torch.utils.data import DataLoader, TensorDataset, Subset, Dataset
from sklearn.model_selection import train_test_split, KFold
from PIL import Image
import shutil
import pandas
from torchvision.io import read_image

cudnn.benchmark = True
plt.ion()   # interactive mode
#https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

In [34]:
def test_model(model, dataset, labels):
    since = time.time()

    # Each epoch has a training and validation phase
    #model.eval()

    running_corrects = 0

    # Iterate over data.
    for idx in range(len(dataset)):
        
        image = Image.open(dataset[idx]).resize((320, 320))
        label = labels[idx]
        if not isinstance(label, list):
            label = [label]
        # zero the parameter gradients
        #optimizer.zero_grad()

        # forward
        output = model(image, size=320)
        print(output.pandas().xyxy[0].sort_values('xmin'))
        predictions = output.pandas().xyxy[0].sort_values('xmin').loc[:, "class"].tolist()
        predictions_char_arr = [str(prediction) if prediction != 10 else 'u' for prediction in predictions]
        predictions_str = "".join(predictions_char_arr)
        print("Number of predictions: ", len(predictions_str), "Prediction: ", predictions_str)
        running_corrects += (predictions == label)
        #output.print()
        #_, preds = torch.max(outputs, 1)
        #print(preds)
        #loss = criterion(output, label)

        # statistics
        #running_loss += loss.item() * inputs.size(0)
        #running_corrects += torch.sum(preds == labels.data)

    #loss = running_loss / dataset_test_size
    acc = running_corrects / len(dataset)

    print(f'Acc: {acc:.4f}')

    time_elapsed = time.time() - since
    print(f'Testing complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')

    return model

In [35]:
preceding_path = "Photos-001"
dataset = [os.path.join(preceding_path, image) for image in os.listdir(preceding_path)]
labels = [[7, 0], [0, 4], [4, 1], [5, 6], [3, 5], [2, 3], [9, 8], [2, 3], [3, 0], [5, 6], [9, 8], [8, 7], [1, 4]]
model_ft = torch.hub.load('ultralytics/yolov5', 'custom', path='best.pt', force_reload=True)

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to C:\Users\justin.rossiter/.cache\torch\hub\master.zip
requirements: YOLOv5 requirements "gitpython" "tqdm>=4.64.0" not found, attempting AutoUpdate...
requirements:  Command 'pip install "gitpython" "tqdm>=4.64.0"  ' returned non-zero exit status 1.
YOLOv5  2022-11-30 Python-3.8.8 torch-1.9.0+cu111 CUDA:0 (NVIDIA GeForce RTX 3090, 24576MiB)

Fusing layers... 
Model summary: 157 layers, 7039792 parameters, 0 gradients
Adding AutoShape... 


In [36]:
test_model(model_ft, dataset, labels)

       xmin      ymin   xmax        ymax  confidence  class name
1  0.695679  0.124344  320.0  319.345703    0.518383      8   pi
0  0.717819  0.124542  320.0  319.694153    0.577186      0    x
Number of predictions:  2 Prediction:  80
       xmin      ymin        xmax        ymax  confidence  class   name
1  0.000000  1.520248  320.000000  313.850586    0.332308      4  equal
0  0.928436  0.097748  319.486053  313.666382    0.475418      0      x
Number of predictions:  2 Prediction:  40
   xmin      ymin   xmax        ymax  confidence  class     name
0   0.0  1.145004  320.0  311.488098    0.564283      7  product
Number of predictions:  1 Prediction:  7
         xmin       ymin        xmax        ymax  confidence  class     name
1    0.000000   2.140503  154.514984  306.800079    0.493954      5  percent
0  148.699951  12.119217  319.481384  304.083862    0.634305      6  partial
Number of predictions:  2 Prediction:  56
         xmin      ymin        xmax        ymax  confidence  

AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  